In [1]:
from transformers import pipeline

pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

Device set to use mps:0


In [2]:
import pandas as pd
pd.options.display.max_colwidth = 500

df = pd.read_csv("open_repair_with_translations.csv")

# Add a new column to provide a little context to the problem
df['problem_with_product'] = df['product_category'] + ': ' + df['problem']

df.head()

/var/folders/mb/f5zh4qyd6sbf171lrlnmff040000gn/T/ipykernel_12096/3483712037.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("open_repair_with_translations.csv")


,id,data_provider,country,partner_product_category,product_category,product_category_id,brand,year_of_manufacture,product_age,repair_status,repair_barrier_if_end_of_life,group_identifier,event_date,problem,problem_eng,problem_with_product
0,anstiftung_23571,anstiftung,DEU,Unterhaltungselektronik ~ Playstation/Spielekonsole,Hi-Fi separates,13,Sony,NaN,NaN,End of life,NaN,35862,2024-07-29,"Keine Funktion mehr, da CD nicht mehr ausgeworfen wird. Nach demontage und öffnen des Blue-Ray Player konnte CD entnommen werden, jedoch die mechanik war offensichtlich zu kompliziert, dass beim zusammenbay die richtige Ursprungsposition nicht gefunden wurde.","No longer works because the CD is no longer ejected. After disassembling and opening the Blue-Ray player, the CD could be removed, but the mechanism was obviously too complicated and the correct original position could not be found when reassembling.","Hi-Fi separates: Keine Funktion mehr, da CD nicht mehr ausgeworfen wird. Nach demontage und öffnen des Blue-Ray Player konnte CD entnommen werden, jedoch die mechanik war offensichtlich zu kompliziert, dass beim zusammenbay die richtige Ursprungsposition nicht gefunden wurde."
1,anstiftung_23570,anstiftung,DEU,Haushaltsgeräte ~ Kaffeevollautomat,Coffee maker,36,Acopino,NaN,NaN,Repairable,NaN,35862,2024-07-29,"Wasser läuft an der Brüheinheit vorbei. Nach mehreren Tassen Kaffee ist der innere Bereich der Brüheinheit voller Wasser. Versucht, Dichtungen die als Ersatzteil bestellt wurden zu erneuern. Leider passten nicht alle. Eine Dichtung am Auslaufstutzen schien undicht, da dort das meiste Wasser nach Brühvorgang heraustropfte.","Water is running past the brewing unit. After several cups of coffee, the inside of the brewing unit is full of water. Tried replacing the seals that were ordered as spare parts. Unfortunately, not all of them fit. A seal on the spout seemed to be leaking, as most of the water dripped out of there after the brewing process.","Coffee maker: Wasser läuft an der Brüheinheit vorbei. Nach mehreren Tassen Kaffee ist der innere Bereich der Brüheinheit voller Wasser. Versucht, Dichtungen die als Ersatzteil bestellt wurden zu erneuern. Leider passten nicht alle. Eine Dichtung am Auslaufstutzen schien undicht, da dort das meiste Wasser nach Brühvorgang heraustropfte."
2,anstiftung_23572,anstiftung,DEU,Haushaltsgeräte ~ Toaster,Toaster,31,AEG,2014.0,10.0,Fixed,NaN,35862,2024-07-29,Temperatur läßt sich nicht mehr regeln. Mechanik des Biemetalls justiert.,Temperature can no longer be regulated. Mechanics of the bimetal adjusted.,Toaster: Temperatur läßt sich nicht mehr regeln. Mechanik des Biemetalls justiert.
3,anstiftung_23596,anstiftung,DEU,Haushaltsgeräte ~ Wassersprudler,Small kitchen item,29,Sodastream,NaN,NaN,End of life,NaN,4571,2024-07-27,Wassersprudler sprudelt nicht mehr. - Druckventil ist defekt. [HE] {Helmut G.},Soda maker no longer bubbles. - Pressure valve is defective. [HE] {Helmut G.},Small kitchen item: Wassersprudler sprudelt nicht mehr. - Druckventil ist defekt. [HE] {Helmut G.}
4,anstiftung_23611,anstiftung,DEU,Haushaltsgeräte ~ Glätteisen,Hair & beauty item,9,Unknown,NaN,NaN,End of life,NaN,4571,2024-07-27,Haarglätter funktioniert seit Sturz auf Boden nicht mehr. - Heizelement defekt. Reparatur nicht rentabel. [HE] {Ha-Jo W.},Hair straightener stopped working after falling on the floor. - Heating element broken. Repair not cost-effective. [HE] {Ha-Jo W.},Hair & beauty item: Haarglätter funktioniert seit Sturz auf Boden nicht mehr. - Heizelement defekt. Reparatur nicht rentabel. [HE] {Ha-Jo W.}


## I ran the below twice to create two samples. I used ChatGPT to classify one of them into 50 specific categories and a few broad categories, and Perplexity to classify another. Then I merged the two, combining similar categories and adding/deleting as needed.

In [8]:
df_sample = df.sample(n=500)

In [9]:
df_sample.to_csv('open_repair_sample2.csv')

In [4]:
import pandas as pd
import csv
from tqdm import tqdm
from transformers import pipeline

pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

categories = [
    "Power Failure", "Battery/Charging Issues", "Connectivity Problems", 
    "Display/Screen Issues", "Audio/Sound Problems", "Button/Switch/Control Issues",
    "Motor/Drive Issues", "Overheating", "Liquid/Water Damage", "Software/Firmware Issues",
    "Physical/Structural Damage", "Wear and Tear/End of Life", "Component Failure (Electronic)",
    "Loose Connections", "Dust/Dirt/Blockage", "Lubrication/Maintenance Needed",
    "Alignment/Calibration Issues", "Timer/Clock Issues", "Heating Problems", 
    "Cooling Problems", "Leaking", "Excessive Noise/Vibration", "Speed Control Problems",
    "Cutting/Grinding Issues (Blades/Bits)", "Dispensing/Filling Issues", 
    "Sealing/Adhesion Problems", "Rust/Corrosion", "Missing Parts", "Incorrect Assembly",
    "Unsafe to Use", "Aesthetic Issues", "Compatibility Issues", "Unknown/Unspecified Problem",
    "Cable/Wire Damage", "Fuse/Circuit Breaker Issues", "Transformer Issues", "Tension Problems",
    "Print Head Issues", "Printer Cartridge Recognition Issue", "CD/DVD Drive or Media Reader Issue",
    "Tape/Ribbon Transport Failure", "Locking Mechanism Failure", 
    "Remote Control or Signal Transmission Issue", "Mechanical Blockage in Moving Parts",
    "Other Issue"
]

def get_category(problem):
    result = pipe(problem, categories)
    return result['labels'][0], result['scores'][0]  # Returns (category, score)

get_category("Los contact. 1 kanaal werkt niet meer. Solderen.")

Device set to use mps:0


('Loose Connections', 0.20434331893920898)

In [5]:
df = df.dropna(subset=['problem'])
df.shape

(163917, 16)

In [6]:
df = df.reset_index()

In [7]:
df = df.drop(columns=['index'])
df.tail()

,id,data_provider,country,partner_product_category,product_category,product_category_id,brand,year_of_manufacture,product_age,repair_status,repair_barrier_if_end_of_life,group_identifier,event_date,problem,problem_eng,problem_with_product
163912,repconn_2881,Repair Connects,BEL,Hair & beauty item,Hair & beauty item,9,Onbekend,NaN,NaN,Repairable,NaN,Maakbaar Leuven,2022-11-16,"Amper gebruikt, sputterde eerst, volgende keer werkte het niet meer, vermoeden probleem met contact",NaN,"Hair & beauty item: Amper gebruikt, sputterde eerst, volgende keer werkte het niet meer, vermoeden probleem met contact"
163913,repconn_2882,Repair Connects,BEL,Hair & beauty item,Hair & beauty item,9,Braun,NaN,NaN,Repairable,NaN,Maakbaar Leuven,2022-08-13,Aan uit knop werkt niet meer,NaN,Hair & beauty item: Aan uit knop werkt niet meer
163914,repconn_2883,Repair Connects,BEL,Hair & beauty item,Hair & beauty item,9,Braun,NaN,NaN,Fixed,NaN,Maakbaar Leuven,2020-06-05,"Stekker werd warm bij gebruik. Stekker afgeknipt, met de bedoeling er een nieuwe op te zetten, en aanhechting te isoleren met krimpkous. Maar, ik twijfel mbt veiligheid. Hele stroomkabel vervangen lukt me niet, owv de gebruikte schroeven, met driehoekige inkeping..",NaN,"Hair & beauty item: Stekker werd warm bij gebruik. Stekker afgeknipt, met de bedoeling er een nieuwe op te zetten, en aanhechting te isoleren met krimpkous. Maar, ik twijfel mbt veiligheid. Hele stroomkabel vervangen lukt me niet, owv de gebruikte schroeven, met driehoekige inkeping.."
163915,repconn_2884,Repair Connects,BEL,Hair & beauty item,Hair & beauty item,9,Calor,NaN,NaN,Fixed,NaN,Maakbaar Leuven,2020-07-20,Het toestel blaast nog en is nog warm. Maar is gevallen en er is een onderdeel (stukje) uitgekomen dus niet zo veilig om nog te gebruiken?,NaN,Hair & beauty item: Het toestel blaast nog en is nog warm. Maar is gevallen en er is een onderdeel (stukje) uitgekomen dus niet zo veilig om nog te gebruiken?
163916,repconn_2885,Repair Connects,BEL,Hair & beauty item,Hair & beauty item,9,Calor,NaN,NaN,Fixed,NaN,Maakbaar Leuven,2021-08-12,De 'trimmer' werkt niet meer. Hij maakt nog geluid dus lijkt het me de connectie.,NaN,Hair & beauty item: De 'trimmer' werkt niet meer. Hij maakt nog geluid dus lijkt het me de connectie.


In [9]:
from tqdm import tqdm
tqdm.pandas()  

In [ ]:
df_subset = df[['problem']].dropna().reset_index(drop=True)  # Keep as DataFrame


output_file = "categorized_results_all.csv"


with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Problem", "Category", "Confidence Score"])  # Ensure correct columns


chunk_size = 10_000
for i in range(0, len(df_subset), chunk_size):
    df_chunk = df_subset.iloc[i:i + chunk_size].copy()  # Copy to avoid Pandas warning


    df_chunk[['Category', 'Confidence Score']] = df_chunk['problem'].progress_apply(
        lambda x: pd.Series(get_category(x))
    )


    df_chunk.to_csv(output_file, mode='a', index=False, header=False)  # Append without headers

print("Processing complete. Results saved to", output_file)


  0%|                                       | 3/10000 [00:07<6:51:19,  2.47s/it]